<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

import plotly
import plotly.express as px

# Добавлено для игнорирования следующего предупреждения:
# UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection.
import warnings
warnings.filterwarnings('ignore')

In [2]:
# connection = psycopg2.connect(dbname = ,user = ,password = ,host = ,port = )

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [10]:
# Во всех запросах данного юнита считаем количество строк в каждой таблице и выводим их в DataFrame.

query_3_1 = '''
select count(*)
from public.vacancies
'''

In [11]:
result_3_1 = pd.read_sql_query(query_3_1, connection)
display(result_3_1['count'][0])

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [12]:
query_3_2 = '''
select count(*)
from public.employers
'''

In [13]:
result_3_2 = pd.read_sql_query(query_3_2, connection)
display(result_3_2['count'][0])

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [14]:
query_3_3 = '''
select count(*)
from public.areas
'''

In [15]:
result_3_3 = pd.read_sql_query(query_3_3, connection)
display(result_3_3['count'][0])

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [16]:
query_3_4 = '''
select count(*)
from public.industries
'''

In [17]:
result_3_4 = pd.read_sql_query(query_3_4, connection)
display(result_3_4['count'][0])

294

На основе предварительного анализа данных можно сделать следующие выводы:
1. Таблица 'Vacancies' хранит в себе данные о 49197 вакансиях;
2. В таблице-справочнике со списком работодателей представлены 23501 работодателей;
3. Таблица-справочник, которая хранит в себе код города и его название содержит 1362 региона;
4. Таблица-справочник вариантов сфер деятельности работодателей включает в себя 294 сферы.

***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [20]:
# Соединяем таблицу вакансий с регионами по id, группируем по региону и выводим требуемые значения с заданной сортировкой.
query_4_1 = '''
select distinct a.id area_id, a.name area_name, count(*) vac_cnt
from public.vacancies v
    join public.areas a on v.area_id = a.id
group by a.id
order by vac_cnt desc
'''

In [21]:
result_4_1 = pd.read_sql_query(query_4_1, connection)
display(result_4_1)

,area_id,area_name,vac_cnt
0,1,Москва,5333
1,2,Санкт-Петербург,2851
2,1002,Минск,2112
3,4,Новосибирск,2006
4,160,Алматы,1892
...,...,...,...
764,6299,Отеген-Батыр,1
765,6313,Палех,1
766,6365,Актогай,1
767,6368,Большие Вязёмы,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [93]:
query_4_2 = '''
select *
from public.vacancies
where salary_from is not null or salary_to is not null
'''

In [95]:
result_4_2 = pd.read_sql_query(query_4_2, connection)
display(result_4_2.shape[0])

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [24]:
# Применяем к значениям зарплатной вилки функцию AVG для вычисления среднего и ROUND для округления.
query_4_3 = '''
select round(avg(salary_from)) salary_from_avg, round(avg(salary_to)) salary_to_avg
from public.vacancies
'''

In [28]:
result_4_3 = pd.read_sql_query(query_4_3, connection)

result_4_3_from_avg = result_4_3['salary_from_avg'][0] # Обращаемся по заданному названию столбца и получаем значение.
result_4_3_to_avg = result_4_3['salary_to_avg'][0]

print(f'Среднее значение для нижней границы зарплатной вилки = {result_4_3_from_avg} руб.')
print(f'Среднее значение для верхней границы зарплатной вилки = {result_4_3_to_avg} руб.')

Среднее значение для нижней границы зарплатной вилки = 71065.0 руб.
Среднее значение для верхней границы зарплатной вилки = 110537.0 руб.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [119]:
# Группируем по schedule и employment, выводим с количеством строк.
query_4_4 = '''
select schedule, employment, count(*)
from public.vacancies
group by schedule, employment
order by count(*) desc
'''

In [120]:
result_4_4 = pd.read_sql_query(query_4_4, connection)
display(result_4_4)

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [12]:
query_4_5 = '''
select experience, count(*)
from public.vacancies
group by experience
order by count(*)
'''

In [13]:
result_4_5 = pd.read_sql_query(query_4_5, connection)
display(result_4_5)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


На основе детального анализа вакансий можно сделать следующие выводы:
1. Наибольшее количество вакансий представлено в Москве, Санкт-Петербург на втором месте, затем идут Минск, Новосибирск, Алматы и т.д.;
2. Более чем у половины представленных вакансий не указана заработная плата;
3. Среднее значение для нижней границы зарплатной вилки составляет 71065 руб., а для верхней границы - 110537 руб.;
4. Большинству вакансий (35367 из 49197) характерен полный рабочий день в сочетании с полной занятостью, на втором месте - удалённая работа с полной занятостью.;
5. Самое популярное значение требуемого опыта - от 1 года до 3 лет (чуть более 50%), далее - от 3 до 6 лет, затем без опыта работы и на последнем месте - более 6 лет. 

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [3]:
# Соединяем вакансии с работодателями, группируем по работодателю и выводим с подсчётом строк.
query_5_1 = '''
select e.name employer, count(*) vac_cnt
from public.vacancies v
    join public.employers e on v.employer_id = e.id
group by e.name
order by count(*) desc
limit 5
'''

In [4]:
result_5_1 = pd.read_sql_query(query_5_1, connection)
display(result_5_1)

,employer,vac_cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [5]:
# Запрос для вывода количества работодателей и вакансий в нём.
query_5_2_1 = '''
select a.name area, count(distinct e.name) emp_cnt, count(*) vac_cnt
from public.vacancies v
    join public.employers e on v.employer_id = e.id
    join public.areas a on v.area_id = a.id
group by a.id
order by 2 desc, 3 desc
'''

# Запрос для нахождения региона в котором на данный момент нет вакансий, и имеющего наибольшее количество работодателей.
query_5_2_2 = '''
select a.name area, count(distinct e.id) emp_cnt
from public.areas a
    left join public.vacancies v on a.id = v.area_id
    left join public.employers e on a.id = e.area
where v.id is null
group by a.id
order by emp_cnt desc
limit 1
'''

In [6]:
result_5_2_1 = pd.read_sql_query(query_5_2_1, connection)
result_5_2_2 = pd.read_sql_query(query_5_2_2, connection)['area'][0]

display(result_5_2_1)
print(f'Среди регионов, в которых нет вакансий, наибольшее количество работодателей в регионе: {result_5_2_2}.')

,area,emp_cnt,vac_cnt
0,Москва,2320,5333
1,Санкт-Петербург,1448,2851
2,Минск,851,2112
3,Новосибирск,802,2006
4,Алматы,798,1892
...,...,...,...
764,Старая Купавна,1,1
765,Камышлов,1,1
766,Электроугли,1,1
767,Сальск,1,1


Среди регионов, в которых нет вакансий, наибольшее количество работодателей в регионе: Россия.


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [7]:
# Посчитаем количество различных регионов с группировкой по работодателю
query_5_3 = '''
select e.name employer, count(distinct v.area_id) areas_cnt
from public.employers e
    join public.vacancies v on e.id = v.employer_id
group by e.id
order by areas_cnt desc
'''

In [8]:
result_5_3 = pd.read_sql_query(query_5_3, connection)
display(result_5_3)

,employer,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [9]:
query_5_4 = '''
select count(*)
from public.employers e
    left join public.employers_industries ei on e.id = ei.employer_id
where industry_id is null
'''

In [10]:
result_5_4 = pd.read_sql_query(query_5_4, connection)['count'][0]
print(f'У {result_5_4} работодателей не указана сфера деятельности.')

У 8419 работодателей не указана сфера деятельности.


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [11]:
# Соединяем работодателей со сферами деятельности и применяем HAVING к сгруппированным по id работодателям.
query_5_5 = '''
select name
from public.employers e
    join public.employers_industries ei on e.id = ei.employer_id
group by id
having count(distinct industry_id) = 4
order by name
'''

In [12]:
result_5_5 = pd.read_sql_query(query_5_5, connection)['name'][2]
print(f'Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности - {result_5_5}.')

Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности - 2ГИС.


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [102]:
query_5_6 = '''
select count(*)
from public.employers_industries ei
    join public.industries i on ei.industry_id = i.id
where name='Разработка программного обеспечения'
'''

In [105]:
result_5_6 = pd.read_sql_query(query_5_6, connection)['count'][0]
print(f'У {result_5_6} работодателей в качестве сферы деятельности указана разработка програмного обеспечения.')

У 3553 работодателей в качестве сферы деятельности указана разработка програмного обеспечения.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [14]:
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'

response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
tables = page.find_all('table') # Ищем все таблицы на странице.
df = pd.read_html(str(tables))[1] # Находим нужную таблицу и получаем DataFrame с городами-миллионниками.

cities_m_list = list(df['Город']) # Получили список городов-миллионников.
print(cities_m_list)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [15]:
# Соединяем вакансии с работодателями и регионами, прописываем требуемые условия, группируем по региону и выводим с количеством строк.
# Во второй части запроса с теми же условиями считаем общее количество строк и выводим его в 'Total', соединяем c помощью UNION.
query_5_7 = f'''
(select a.name area, count(*)
from public.vacancies v
    join public.employers e on v.employer_id = e.id
    join public.areas a on v.area_id = a.id
where e.name = 'Яндекс' and a.name in {tuple(cities_m_list)}
group by a.name)

union all

(select 'Total', count(*)
from public.vacancies v
    join public.employers e on v.employer_id = e.id
    join public.areas a on v.area_id = a.id
where e.name = 'Яндекс' and a.name in {tuple(cities_m_list)})

order by count
'''

In [16]:
result_5_7 = pd.read_sql_query(query_5_7, connection)
display(result_5_7)

На основе анализа работодателей можно сделать вывод, что наибольшее количество вакансий в нашей базе данных представлено работодателем Яндекс и с большим отрывом. Яндекс также лидирует по количеству различных регионов, в которых он публикует свои вакансии. В рамках анализа подробнее рассмотрели вакансии Яндекса и представили распределение их количества по городам-миллионникам.

Также стоит обратить внимание на то, что у многих работодателей не указано ни одной сферы деятельности (у 8419 из 23501), хотя может быть указано несколько.

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [3]:
# Прописываем в WHERE заданные условия и считаем строки.
query_6_1 = '''
select count(*)
from public.vacancies
where lower(name) like '%data%' or lower(name) like '%данн%' 
'''

In [4]:
result_6_1 = pd.read_sql_query(query_6_1, connection)['count'][0]
print(f'{result_6_1} вакансия имеет отношение к данным.')

1771 вакансия имеет отношение к данным.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [16]:
query_6_2 = '''
select *
from public.vacancies
where
    (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    
    and
    
    (lower(name) like '%junior%'
    or experience = 'Нет опыта'
    or employment = 'Стажировка') 
'''

In [17]:
result_6_2 = pd.read_sql_query(query_6_2, connection).shape[0]
print(f'В базе данных есть {result_6_2} подходящяя вакансия для начинающиего дата-сайентиста.')

В базе данных есть 51 подходящяя вакансия для начинающиего дата-сайентиста.


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [11]:
query_6_3 = '''
select *
from public.vacancies
where
    (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    
    and
    
    (upper(key_skills) like '%SQL%'
    or lower(key_skills) like '%postgres%')
'''

In [13]:
result_6_3 = pd.read_sql_query(query_6_3, connection).shape[0]
print(f'В базе данных есть {result_6_3} вакансия для DS, где в качестве ключевого навыка указан SQL или postgres.')

В базе данных есть 201 вакансия для DS, где в качестве ключевого навыка указан SQL или postgres.


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [16]:
query_6_4 = '''
select *
from public.vacancies
where
    (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    
    and
    
    (lower(key_skills) like '%python%')
'''

In [17]:
result_6_4 = pd.read_sql_query(query_6_4, connection).shape[0]
print(f'В базе данных есть {result_6_4} вакансия для DS, где в качестве ключевого навыка указан Python.')

В базе данных есть 351 вакансия для DS, где в качестве ключевого навыка указан Python.


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [80]:
query_6_5 = '''
select (length(key_skills) - length(replace(key_skills, CHR(9), ''))) count
from public.vacancies
where
    (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    
    and
    
    (key_skills is not null)
'''

In [88]:
result_6_5 = pd.read_sql_query(query_6_5, connection).apply(lambda x: x + 1)['count'].mean()
# Прибавляем единицу, так как ключевых навыков на единицу больше, чем разделителей.

print(f'В среднем указывают {round(result_6_5, 2)} ключевых навыков в вакансиях для DS.')

В среднем указывают 6.41 ключевых навыков в вакансиях для DS.


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [36]:
# Идентично задаём условия, с помощью COALESCE выводим требуемые значения, заменяя NULL на значение противоположной границы зарплатное вилки.
query_6_6 = '''
select experience, ((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2) salary
from public.vacancies
where
    (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not like '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    
    and
    
    (salary_from is not null or salary_to is not null)
'''

In [40]:
# Группируем полученный DF по требуемому опыту и выводим округлённое среднее значение заработной платы.
result_6_6 = pd.read_sql_query(query_6_6, connection).groupby('experience')['salary'].mean().round()
display(result_6_6)

experience
Нет опыта              74643.0
От 1 года до 3 лет    139675.0
От 3 до 6 лет         243115.0
Name: salary, dtype: float64

В данном юните были рассмотрены вакансии, которые можно отнести к Data Science.

В нашей базе данных таких вакансий - 1771, а начинающему дата-сайентисту подходят 51 из них.

Далее подробнее были рассмотрены ключевые навыки свойственные этим вакансиям. В 201 вакансии в качестве ключевого навыка указан SQL или Postgres, а Python в 351.
В среднем в вакансиях для DS указывают 6 - 7 ключевых навыков.

Также рассмотрели средние значения заработной платы по таким вакансиям. Соискателям без опыта работы предлагают в среднем 75 тыс. руб., специалистам с опытом от 1 года до 3 лет - 140 тыс. руб., а с опытом от 3 до 6 лет - 243 тыс. руб.

***

# Общий вывод по проекту

В рассматриваемой базе данных содержится информация о 49197 вакансиях.

Наибольшее количество вакансий представлено в Москве, Санкт-Петербурге, Минске, Новосибирске и Алматы.
Ранее мы определили, что нижняя граница зарплатной вилки в среднем составляет 71 тыс. руб., а верхняя - 110 тыс. руб.
Посмотрим как данные характеристики изменяются в зависимости от региона в разрезе десяти лидирующих по количеству вакансий городов:


In [7]:
query_7_1 = '''
select a.name area, round(avg(salary_from)) avg_salary_from, round(avg(salary_to)) avg_salary_to
from public.vacancies v
    join public.areas a on v.area_id = a.id
group by a.name
order by count(*) desc
limit 10
'''

result_7_1 = pd.read_sql_query(query_7_1, connection)
display(result_7_1)

,area,avg_salary_from,avg_salary_to
0,Москва,113409.0,160594.0
1,Санкт-Петербург,89892.0,127249.0
2,Минск,68040.0,133797.0
3,Новосибирск,91272.0,137573.0
4,Алматы,53498.0,96238.0
5,Екатеринбург,77489.0,120404.0
6,Нижний Новгород,83734.0,126921.0
7,Казань,76117.0,119942.0
8,Краснодар,78483.0,117144.0
9,Самара,67885.0,101157.0


Как и следовало ожидать, в Москве средние значения зарплатной вилки сильно выше значений без разбивки по регионам. С уменьшением количества вакансий (что можно предположить в свою очередь связано с уменьшением численности населения) значения приближаются к усреднённым по всем регионам. Стоит обратить внимание на то, что в выборке встречаются не только города России и средние значения зарплатной вилки для них значительно ниже, чем у схожих по количеству вакансий регионов РФ. 

В рамках анализа вакансий мы определили, что большинству (72%) вакансий характерено сочетание полного рабочего дня с полной занятостью, а на втором месте в 16% случаев встречается сочетание удалённой работы с полной занятостью. Можно продположить, что сейчас на рынке труда удалённая работа набирает популярность и чтобы это проверить можно провести дополнительное исследование, при наличии данных за различные промежутки времени. 

Также мы рассмотрели наши вакансии в разрезе требуемого опыта и определили, что в более чем 50% случаев его показатель - от 1 года до 3 лет. На представленном ниже графике показано полное распределение. 

In [24]:
px.pie(result_4_5, 'experience', 'count', width=600, height=400, title='Распределение вакансий по требуемому опыту')

В нашем исследовании мы подробнее остановились на вакасиях, которые можно отнести к Data Science.

Таких вакансий мы обнаружили 1771, среди которых 51 подходит начинающему дата-сайентисту (DS).
В 20% вакансий для DS в качестве ключевого навыка указан Python, а SQL или Postgres в 10% случаев.
Заработная плата для соискателей без опыта работы в среднем составляет 75 тыс. руб., с опытом от 1 года до 3 лет - 140 тыс. руб., а с опытом от 3 до 6 лет - 243 тыс. руб.

В рамках проведённого анализа работодателей мы определили, что наибольшее количество вакансий в нашей базе данных представлено работодателем 'Яндекс' и он лидирует с большим отрывом, а также показали распределение опубликованных им вакансий по городам-миллионникам России.

В заключение, дополнительно рассмотрим вакансии данного работодателя на предмет отношения их к Data Science.

In [7]:
query_7_2 = '''
select *
from public.vacancies v
    join public.employers e on v.employer_id = e.id
where
    (lower(v.name) like '%data scientist%'
    or lower(v.name) like '%data science%'
    or lower(v.name) like '%исследователь данных%'
    or (v.name like '%ML%' and v.name not like '%HTML%')
    or lower(v.name) like '%machine learning%'
    or lower(v.name) like '%машинн%обучен%')
    
    and
    
    (e.name = 'Яндекс')
'''

result_7_2 = pd.read_sql_query(query_7_2, connection).shape[0]
print(f"В базе данных есть {result_7_2} вакансий для DS, где работодателем является 'Яндекс'.")

В базе данных есть 9 вакансий для DS, где работодателем является 'Яндекс'.
